# simple nn model fitting

Notebook will run a very simple nn on the training points

In [7]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
#define network class
#used some help from https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

class Net(nn.Module):
    def __init__(self, input_size, output_size = 1):
        super(Net, self).__init__()
        #linear layers to keep decrease the number of nodes
        self.layer1 = nn.Linear(input_size, int(input_size/2))
        self.layer2 = nn.Linear(int(input_size/2), int(input_size/4))
        self.layer3 = nn.Linear(int(input_size/4), int(input_size/8))
        self.layer4 = nn.Linear(int(input_size/8), int(input_size/16))
        self.last = nn.Linear(int(input_size/16), output_size)

    # x represents our data
    def forward(self, x):
        #these are a bunch of linear layers with relu activation
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = self.last(x)
        return x
    
    def classification_error(self, inputs, outputs):
        count = 0
        pred = self.forward(inputs.T)
#         loss = criterion(pred, outputs)
        
        #count number predicted correctly
        for i in range(len(outputs)):
            idx_output = torch.argmax(pred[i])
            idx_truth = torch.argmax(outputs[i])
            if idx_output == idx_truth:
                count += 1
            else:
                pass #do nothing if not equal
        print("percent predicted correctly is ", 100*count/len(outputs), '%')
        return 100*count/len(outputs)
        

        

In [9]:
#import training data
train_data = torch.tensor(np.load('data/train_data.npy').T).float()[:1000]
labels = torch.tensor(np.load('data/train_labels.npy')).float()
n_dim, n_training_points = train_data.shape
print("n_dim = ", n_dim)
print('n_training_points = ', n_training_points )

#get testing data
test_data = torch.tensor(np.load('data/test_data.npy').T).float()[:1000]
test_labels = torch.tensor(np.load('data/test_labels.npy')).float()
_, n_testing_points = test_data.shape
print("n_dim = ", n_dim)
print('n_training_points = ', n_testing_points )
print(test_data)


#one hot encode the genre labels
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(torch.cat((labels, test_labels)).unsqueeze(1)) #takes a 2d array as input and labels is 1d; need labels for all training and testing points
print(len(enc.categories_))
labels = enc.transform(labels.unsqueeze(1)).toarray()
labels = torch.tensor(labels).float()
print(labels.shape)
_, n_categories = labels.shape #number of categories is out output size for our network


#encode testing labels
test_labels = enc.transform(test_labels.unsqueeze(1)).toarray()
test_labels = torch.tensor(test_labels).float()
print(test_labels.shape)


n_dim =  1000
n_training_points =  50
n_dim =  1000
n_training_points =  25
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0077, -0.1812, -0.1608,  ...,  0.2518,  0.1378,  0.1512],
        [-0.0131, -0.1836, -0.1614,  ...,  0.2579,  0.1327,  0.1578],
        [-0.0178, -0.1850, -0.1577,  ...,  0.2488,  0.1278,  0.1573]])
1
torch.Size([50, 18])
torch.Size([25, 18])


In [10]:
#define the model, optimizer and loss function
model = Net(n_dim, n_categories)
print(model)
optimizer = torch.optim.Adam(model.parameters()) #adam is a "better" version of stochastic gradient decent
criterion = nn.MSELoss()

Net(
  (layer1): Linear(in_features=1000, out_features=500, bias=True)
  (layer2): Linear(in_features=500, out_features=250, bias=True)
  (layer3): Linear(in_features=250, out_features=125, bias=True)
  (layer4): Linear(in_features=125, out_features=62, bias=True)
  (last): Linear(in_features=62, out_features=18, bias=True)
)


In [11]:
max_epochs = 200
max_test_loss = 1e10
for epoch in range(max_epochs):
    optimizer.zero_grad()   # zero the gradient buffers
    output = model(train_data.T)#put through model
    loss = criterion(output, labels)#find loss
    #find test loss
    test_output = model(test_data.T) 
    test_loss = criterion(test_output, test_labels)
    if(test_loss< max_test_loss):
        max_test_loss = test_loss
        torch.save(model, "model.pt")
        print("model saved at epoch ", epoch)
    loss.backward() #do the back propogation
    optimizer.step()    # Does the update
    
    print("loss = ", loss, ": test_loss = ", test_loss)
    

model saved at epoch  0


C:\Users\16kat\anaconda3\lib\site-packages\torch\autograd\__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


loss =  tensor(0.0581, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0600, grad_fn=<MseLossBackward>)
model saved at epoch  1
loss =  tensor(0.0561, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0588, grad_fn=<MseLossBackward>)
model saved at epoch  2
loss =  tensor(0.0543, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0579, grad_fn=<MseLossBackward>)
model saved at epoch  3
loss =  tensor(0.0525, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0572, grad_fn=<MseLossBackward>)
model saved at epoch  4
loss =  tensor(0.0506, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0566, grad_fn=<MseLossBackward>)
model saved at epoch  5
loss =  tensor(0.0487, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0564, grad_fn=<MseLossBackward>)
loss =  tensor(0.0468, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0568, grad_fn=<MseLossBackward>)
loss =  tensor(0.0453, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0580, grad_fn=<MseLossBackward>)
loss =  tensor(0.0440, g

loss =  tensor(0.0123, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1300, grad_fn=<MseLossBackward>)
loss =  tensor(0.0121, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1303, grad_fn=<MseLossBackward>)
loss =  tensor(0.0119, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1311, grad_fn=<MseLossBackward>)
loss =  tensor(0.0117, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1324, grad_fn=<MseLossBackward>)
loss =  tensor(0.0115, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1319, grad_fn=<MseLossBackward>)
loss =  tensor(0.0114, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1319, grad_fn=<MseLossBackward>)
loss =  tensor(0.0112, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1326, grad_fn=<MseLossBackward>)
loss =  tensor(0.0111, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1344, grad_fn=<MseLossBackward>)
loss =  tensor(0.0111, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1352, grad_fn=<MseLossBackward>)
loss =  tensor(0.0110, grad_fn=<MseLo

loss =  tensor(0.0047, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2262, grad_fn=<MseLossBackward>)
loss =  tensor(0.0047, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2234, grad_fn=<MseLossBackward>)
loss =  tensor(0.0047, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2254, grad_fn=<MseLossBackward>)
loss =  tensor(0.0046, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2263, grad_fn=<MseLossBackward>)
loss =  tensor(0.0046, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2238, grad_fn=<MseLossBackward>)
loss =  tensor(0.0046, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2275, grad_fn=<MseLossBackward>)
loss =  tensor(0.0045, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2265, grad_fn=<MseLossBackward>)
loss =  tensor(0.0045, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2254, grad_fn=<MseLossBackward>)
loss =  tensor(0.0044, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2289, grad_fn=<MseLossBackward>)
loss =  tensor(0.0044, grad_fn=<MseLo

In [12]:
#load best model
model = torch.load('model.pt')

In [19]:
#find classification errors
print("train error: ")
train_error = model.classification_error(train_data, labels)
print("test error: ")
test_error = model.classification_error(test_data, test_labels)

train error: 
percent predicted correctly is  52.0 %
test error: 
percent predicted correctly is  16.0 %
